# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import scipy.stats as st
from scipy.stats import linregress
from pprint import pprint
from matplotlib import figure

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# variable for the path
weather_data_path = "../WeatherPy/output_data/weather.csv"

# read the data / study results

weather_data = pd.read_csv(weather_data_path)

# create a new dataframe
weather_data_df = weather_data
weather_data_df


,Unnamed: 0,City,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lng,Country
0,0,Half Moon Bay,61.00,71,1,8.05,37.4636,-122.4286,US
1,1,Rocha,63.48,94,3,4.12,-34.4833,-54.3333,UY
2,2,Dauphin,-22.00,63,1,12.66,51.1494,-100.0502,CA
3,3,Rikitea,78.78,69,73,10.56,-23.1203,-134.9692,PF
4,4,Esik,37.40,70,90,8.95,43.3552,77.4525,KZ
...,...,...,...,...,...,...,...,...,...
568,568,Conde,78.80,83,20,4.61,-7.2597,-34.9075,BR
569,569,Libertador General San Martín,73.96,65,20,3.18,-23.8064,-64.7876,AR
570,570,Lewistown,28.99,58,1,2.04,40.5992,-77.5714,US
571,571,Batagay-Alyta,-23.49,84,100,1.92,67.8006,130.4114,RU


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# configure gmaps api key
gmaps.configure(api_key=g_key)

# lat and lng locations / humidty
locations = weather_data_df[['Lat','Lng']]
humidity = weather_data_df['Humidity (%)'].astype(float)

# plot heatmap
fig = gmaps.figure()

# create heatmap layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                             dissipating = False, max_intensity = 100,
                             point_radius = 1)

# add layer
fig.add_layer(heat_layer)

# display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [4]:
# # hybrid map
# fig = gmaps.figure(map_type="HYBRID")

# # create heat layer
# heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
#                                 dissipating = False, max_intensity = 100,
#                                 point_radius = 1)

# fig.add_layer(heat_layer)
# fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# create a new dataframe
# set the idea ltemp
ideal_city_df = weather_data_df.loc[(weather_data_df['Max Temp (F)']>=70)]
ideal_city_df

# set the wind speed
ideal_city_df = ideal_city_df.loc[(ideal_city_df['Wind Speed (mph)']<=15)]
ideal_city_df

# set the cloudiness
ideal_city_df = ideal_city_df.loc[(ideal_city_df['Cloudiness (%)']<=25)]
ideal_city_df

# set the humidity
ideal_city_df = ideal_city_df.loc[(ideal_city_df['Humidity (%)']<50)]
ideal_city_df

# drop null values
ideal_city_df.dropna()
ideal_city_df

,Unnamed: 0,City,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lng,Country
20,20,Mataura,81.86,44,5,1.34,-46.1927,168.8643,NZ
61,61,Filingué,71.96,21,10,7.83,14.3521,3.3168,NE
158,158,Kailua,77.00,47,1,10.36,21.4022,-157.7394,US
164,164,‘Āhuimanu,77.00,47,1,10.36,21.4447,-157.8378,US
170,170,Makakilo City,78.80,44,1,14.97,21.3469,-158.0858,US
287,287,Tuatapere,73.13,44,15,4.32,-46.1333,167.6833,NZ
292,292,San Juan,84.15,32,0,10.94,-31.5375,-68.5364,AR
324,324,Leeton,78.01,46,0,11.50,-34.5667,146.4000,AU
325,325,Iquique,73.40,49,0,9.22,-20.2208,-70.1431,CL
403,403,Te Anau,75.51,39,0,6.08,-45.4167,167.7167,NZ


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_city_df
hotel_df['Hotel Name'] = ""
hotel_df

# set variables
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
target_radius = 5000
target_type = 'lodging'

# set up a paremters dictionary
params = {'key':g_key,'radius':target_radius, 'type':target_type}


In [8]:
# find hotels for loop
for index, row in hotel_df.iterrows():
    # grab location from hotel_df 
    Lat = row['Lat']
    Lng = row['Lng']
    
    params['location'] = f'{Lat},{Lng}'
    
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    print(f'Retrieving Results for Index') 
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']

    # saves hotel
    try:                
        hotel_df.loc[index,'Hotel Name'] = results[0]['name']    
        
        print(f'Closest hotel is {Lat},{Lng}: {results[0]["name"]}')        
        
    except (KeyError, IndexError):
        print("missing field/result... skipping.")

print("------------")
hotel_df

Retrieving Results for Index
Closest hotel is -46.1927,168.8643: Ellie's Villa
Retrieving Results for Index
missing field/result... skipping.
Retrieving Results for Index
Closest hotel is 21.4022,-157.7394: Sheffield House Bed & Breakfast
Retrieving Results for Index
Closest hotel is 21.4447,-157.8378: Paradise Bay Resort
Retrieving Results for Index
Closest hotel is 21.3469,-158.0858: Marriott's Ko Olina Beach Club
Retrieving Results for Index
Closest hotel is -46.1333,167.6833: Ron and Tony's Bed & Breakfast
Retrieving Results for Index
Closest hotel is -31.5375,-68.5364: Hotel Provincial
Retrieving Results for Index
Closest hotel is -34.5667,146.4: Historic Hydro Motor Inn
Retrieving Results for Index
Closest hotel is -20.2208,-70.1431: Backpacker`s Hostel Iquique
Retrieving Results for Index
Closest hotel is -45.4167,167.7167: Kingsgate Hotel Te Anau
------------


,Unnamed: 0,City,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lat,Lng,Country,Hotel Name
20,20,Mataura,81.86,44,5,1.34,-46.1927,168.8643,NZ,Ellie's Villa
61,61,Filingué,71.96,21,10,7.83,14.3521,3.3168,NE,
158,158,Kailua,77.00,47,1,10.36,21.4022,-157.7394,US,Sheffield House Bed & Breakfast
164,164,‘Āhuimanu,77.00,47,1,10.36,21.4447,-157.8378,US,Paradise Bay Resort
170,170,Makakilo City,78.80,44,1,14.97,21.3469,-158.0858,US,Marriott's Ko Olina Beach Club
287,287,Tuatapere,73.13,44,15,4.32,-46.1333,167.6833,NZ,Ron and Tony's Bed & Breakfast
292,292,San Juan,84.15,32,0,10.94,-31.5375,-68.5364,AR,Hotel Provincial
324,324,Leeton,78.01,46,0,11.50,-34.5667,146.4000,AU,Historic Hydro Motor Inn
325,325,Iquique,73.40,49,0,9.22,-20.2208,-70.1431,CL,Backpacker`s Hostel Iquique
403,403,Te Anau,75.51,39,0,6.08,-45.4167,167.7167,NZ,Kingsgate Hotel Te Anau


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
# marker_locations = gmaps.marker_layer(locations)

# info boxes
markers = gmaps.marker_layer(locations, info_box_content= hotel_info)

# add the marker layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))